In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
import tensorflow.keras.metrics
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import History 
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV, RFE
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [2]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
df = df[~df["PCL_Strict3"].isna()]


In [3]:
features = ["cd_risc1", "PCL2", "PCL1", "T1bias", "T1Acc1n", "T1Acc1t"]

In [7]:
#bad_features = ["T1ETBE", "T1bias", "state1", "state2", "trait1", "trait2", "phq1", "phq2", "PCL1", "PCL2",
 #               "denial1", "substance_use1", "self_blame1", "denial2", "substance_use2","self_blame2",
  #                       "trauma_history8_1"]
numerical_features = [ "T1ETBE", "T1Acc1t", "T1Acc1n", "T1bias", "T2Acc1t","T2Acc1n", "T2bias", "state1", "state2",
                        "trait1", "trait2", "lot1", "lot2", "phq1", "phq2", "cd_risc1", "PCL1", "PCL2"]
categorical_features = ["age", "highschool_diploma",  "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1", "trait1",
               "state1", "PCL1",  "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2", "PCL_Broad2", 
                 "PCL_Strict2", "cd_risc1", "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                 "religion1", "emotional_support1","instrumental_support1", "self_distraction1", "denial1", 
               "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2", "instrumental_support2", 
                 "self_distraction2", "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                 "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Ranked", "COMT_Hap1_recode", 
               "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale", "Sephar_scale", "Unknown"]

#df['bad_features'] = (df > df.mean())[bad_features].sum(axis=1)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])

# pca = PCA(n_components = 1)
# df["pcls"] = pca.fit_transform(df[["PCL1",  "PCL_Broad1", "PCL_Strict1"]])
# df.drop(["PCL1",  "PCL_Broad1", "PCL_Strict1"], axis=1, inplace=True)
# df["pcls2"] = pca.fit_transform(df[["PCL2",  "PCL_Broad2", "PCL_Strict2"]])
# df.drop(["PCL2",  "PCL_Broad2", "PCL_Strict2"], axis=1, inplace=True)
# features.append("pcls2")

# features.remove("PCL1")
# features.remove("PCL_Broad1")
# features.remove("PCL_Strict1")


# features.remove("PCL2")
# features.remove("PCL_Broad2")
# features.remove("PCL_Strict2")

X = df[features]


ss = StandardScaler()
X = ss.fit_transform(X)


Y = df["PCL_Strict3"]

In [5]:
unsup = 1

if unsup:

    rand_item = eval("['pcls2', 'positive_reframing1', 'trauma_history8_1', 'lot2', 'T1bias', 'emotional_support1', 'self_blame1', 'pcls2']")

    pca = PCA(n_components = 2)
    data_transformed = df[rand_item]
    data_transformed = pca.fit_transform(data_transformed)

    db = DBSCAN(eps=0.5, min_samples=10, leaf_size=30).fit(X)
    X = np.hstack((db.labels_.reshape(-1,1), X))


    rand_item = eval("['instrumental_support1', 'COMT_Hap1_recode', 'dyslexia', 'HL_MAOA', 'lot1', 'active_coping2', 'ADHD']")

    pca = PCA(n_components = 2)
    data_transformed = df[rand_item]
    data_transformed = pca.fit_transform(data_transformed)

    db = DBSCAN(eps=0.7, min_samples=10, leaf_size=30).fit(X)
    X = np.hstack((db.labels_.reshape(-1,1), X))


    rand_item = eval("['COMT_Hap1_LvsMH', 'venting2', 'positive_reframing2', 'COMT_Ranked', 'behavioral_disengagement2', 'trait1', 'HL_MAOA', 'HL_MAOA']")

    pca = PCA(n_components = 2)
    data_transformed = df[rand_item]
    data_transformed = pca.fit_transform(data_transformed)

    db = DBSCAN(eps=0.5, min_samples=10, leaf_size=30).fit(X)
    X = np.hstack((db.labels_.reshape(-1,1), X))


    rand_item = eval("['pcls2', 'emotional_support2', 'pcls2', 'emotional_support1', 'COMT_Hap1_LvsMH', 'HML_FKBP5', 'lot1', 'COMT_Hap1_LvsMH', 'COMT_Hap2_recode', 'state1']")

    pca = PCA(n_components = 2)
    data_transformed = df[rand_item]
    data_transformed = pca.fit_transform(data_transformed)

    db = DBSCAN(eps=0.6, min_samples=10, leaf_size=30).fit(X)
    X = np.hstack((db.labels_.reshape(-1,1), X))


    rand_item = eval("['phq2', 'substance_use1', 'instrumental_support2', 'substance_use1', 'trait1']")

    pca = PCA(n_components = 2)
    data_transformed = df[rand_item]
    data_transformed = pca.fit_transform(data_transformed)

    db = DBSCAN(eps=0.5, min_samples=5, leaf_size=30).fit(X)
    X = np.hstack((db.labels_.reshape(-1,1), X))


KeyError: "['pcls2'] not in index"

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
#X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, random_state=271828, stratify=y_train)


In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True)
cvscores = []
y_train_2 = np.array(y_train)
X_train_2 = np.array(X_train)

for num_layers in [1, 3]:
    for smote_rate in [0.3, 1, 0.7]:
        for k_neighbors in [20, 5]:
            for each_layer in [10, 5]:
                num_layers = num_layers
                first_layer = 10
                each_layer = each_layer
                num_smote = 1
                loops = loops
                scores_f = []
                scores_r = []
                scores_p = []
                for train, test in kfold.split(X_train_2, y_train_2):
                    print("num_layers", num_layers, "\nsmote_rate", smote_rate, 
                          "\neach_layer", each_layer, "\nnum_smote", num_smote, "\nk_neighbors", k_neighbors)
                    sm = SMOTE(smote_rate, k_neighbors= k_neighbors)
                    X_train_res = X_train_2[train]
                    y_train_res = y_train_2[train]
                    
                    
                    X_train_res, y_train_res = sm.fit_sample(X_train_res, y_train_res.ravel())

                    # create model
                    n_cols = X_train_res.shape[1]
                    model = Sequential()
                    model.add(Dense(first_layer, activation='elu', input_dim = n_cols))
                    model.add(Dropout(0.5))

                    for i in range(num_layers):
                        model.add(Dense(each_layer, activation='elu'))
                        model.add(Dropout(0.5))

                    model.add(Dense(1, activation='sigmoid'))

                    model.compile(optimizer='adam', 
                                  loss='binary_crossentropy')


                    # Fit the model
                    callbacks = [EarlyStopping(monitor='val_loss', patience=1)]
                    model.fit(X_train_res, y_train_res, epochs = 300, validation_split = .1, verbose=0,
                              callbacks=callbacks, class_weight = {1:1, 0:5})
                    # evaluate the model
                    y_pred =  model.predict(X_train_2[test])
                    y_train_pred =  model.predict(X_train_2[train])
                    #preds.extend(y_pred)
                    #trues.extend(y_train_2[test])
                    y_pred = y_pred>0.5
                    s_f = f1_score(y_train_2[test], y_pred)
                    s_p = precision_score(y_train_2[test], y_pred)
                    s_r = recall_score(y_train_2[test], y_pred)
                    #print("\tscores f1", (s_f))
                    #print("\tscores p", (s_p))
                    #print("\tscores r", (s_r))
                    s = f1_score(y_pred,y_train_2[test])
                    #print(s)
                    scores_f.append(s_f)
                    scores_p.append(s_p)
                    scores_r.append(s_r)

                    
                    y_train_pred = (y_train_pred) > 0.5
                    train_s_f = f1_score(y_train_2[train], y_train_pred)
                    train_s_p = precision_score(y_train_2[train], y_train_pred)
                    train_s_r = recall_score(y_train_2[train], y_train_pred)
                    #print("\tscores f1 train", (train_s_f))
                    #print("\tscores p train", (train_s_p))
                    #print("\tscores r train", (train_s_r))
                print("mean scores  f", np.mean(scores_f))
                print("mean scores  p", np.mean(scores_p))
                print("mean scores  r", np.mean(scores_r))

num_layers 1 
smote_rate 0.3 
each_layer 10 
num_smote 1 
k_neighbors 20
num_layers 1 
smote_rate 0.3 
each_layer 10 
num_smote 1 
k_neighbors 20
num_layers 1 
smote_rate 0.3 
each_layer 10 
num_smote 1 
k_neighbors 20


In [15]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cvscores = []
y_train_2 = np.array(y_train)
X_train_2 = np.array(X_train)

#eval
pres=[]
rs = []
for train, test in kfold.split(X_train_2, y_train_2):

    rfe =  RFE(RandomForestClassifier(n_estimators=100), 7).fit(X_train_2[train], y_train_2[train])
    print(rfe.ranking_)
    X_train_rfe = rfe.transform(X_train_2[train])
    X_test_rfe = rfe.transform(X_train_2[test])
    
    sm = SMOTE(0.2, k_neighbors= 8)
    X_train_res, y_train_res = sm.fit_sample(X_train_rfe, y_train_2[train].ravel())

    model_number = 4

        
    if model_number == 4:
        # create model
        n_cols = X_train_res.shape[1]
        model = Sequential()


        
        model.add(Dense(15, activation='elu', input_dim = n_cols))
        model.add(Dropout(0.5)) 


        model.add(Dense(1, activation='sigmoid'))

        model.compile(optimizer='adam', loss='binary_crossentropy')


        # Fit the model
        callbacks = [EarlyStopping(monitor='val_loss', patience=5)]
        
        # Fit the model
        model.fit(X_train_res, y_train_res, epochs = 500, verbose=0, 
                  validation_split = .1, class_weight = {0:12, 1:3}, callbacks=callbacks)

        # evaluate the model                    
        y_pred =  model.predict(X_test_rfe)
        y_pred = y_pred > 0.5

        
    s = precision_score(y_train_2[test], y_pred)
    print("\n\nprecision\n", s)
    pres.append(s)
    
    r = recall_score(y_train_2[test], y_pred)
    print("recall\n", r)
    rs.append(r)
    
print("\n\nmean precision", sum(pres)/len(pres))
print("\n\nmean recall", sum(rs)/len(rs))

[54 53 52 51 50  9 41 48 20 35 22  6  5  3  2  1  1  1  1  1  1 24 45 43
 36 37 10 46 39 26 19 32 42 17 38 33 34 13 31 25 23  8 29 14 28 12  7 27
 21 11 15 40 16 18 47 44 49 30  4  1]


precision
 0.3333333333333333
recall
 0.625
[54 53 52 51 50  8 44 46 24 13 36  1  2 17  7  5  1  1  1  1  1 47 40 38
 39 32 20 41 29 28 37 18 43 12 21 27 35  9 23 16 31 19 14  4  6 10 22 25
 30 11 34 42 26 45 48 33 49 15  3  1]


precision
 0.14285714285714285
recall
 0.25
[54 53 52 51 50  6 28 48 46 41 21  1  7  5  3  4  1  2  1  1  1 17 44 47
 40 31 22 32 24 29 27 36 38 16 23 45 33 25 30 15 39 26 14 11 13 18  9 37
 20 12 34 35  8 19 43 42 49 10  1  1]


precision
 0.15625
recall
 0.625
[54 53 52 51 50  5 23 48 44 37 36  3  1  2  4  1  1  1  1  1  6 34 45 42
 38 21 12 33 32 25 27 19 43 14 41 30 39  9 24 13 28 10 26  8 20 11 16 29
 17 22 15 40 18 31 47 35 49 46  7  1]


precision
 0.1111111111111111
recall
 0.125
[54 53 52 51 50 21 46 47 19 37 33  1  7  6  4  1  1  3  1  1  1 28 42 43
 38 30 15 39 24 31

In [9]:
##

# RFE = 5 
## unsup 1 model 4 - 25/20/15
## p - 0.82
## r - 0.12


# RFE = 5 
## unsup 1 model 4 - 25/20/15    w - 10/20
## p - 0.82
## r - 0.12


# RFE = 5 
## unsup 1 model 4 - 25/20/15    w - 10/15
## p - 0.8
## r - 0.12


# RFE = 4
## unsup 1 model 4 - 25/20/15    w - 10/15
## p - 0.8
## r - 0.12

In [10]:
# RFE = 3

## unsup 1 model 4 - 
## p - 0.857
## r - 0.0995
## unsup 1 model 0 - 
## p - 0.86
## r - 0.099

## unsup 0 model 4 - 
## p - 0.839
## r - 0.0995
## unsup 0 model 0 - 
## p - 0.725
## r - 0.101


# RFE = 13

## unsup 1 model 4 - 
## p - 0.52
## r - 0.14

## unsup 0 model 4 - 
## p - 0.6
## r - 0.15


# RFE = 7

## unsup 1 model 4 - 
## p - 0.67
## r - 0.14


# RFE = 5

## unsup 1 model 4 - 
## p - 0.67
## r - 0.14

In [42]:

rfe =  RFE(RandomForestClassifier(n_estimators=100), 7).fit(X_train_2, y_train_2)

X_train_rfe = rfe.transform(X_train_2)
X_test_rfe = rfe.transform(X_test_2)
    
sm = SMOTE(random_state=27)
X_train_res, y_train_res = sm.fit_sample(X_train_rfe, y_train_2.ravel())

# create model
n_cols = X_train_res.shape[1]
model = Sequential()
    
model.add(Dense(10, activation='elu', input_dim = n_cols))
model.add(Dropout(0.5))


model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')


# Fit the model
model.fit(X_train_res, y_train_res, epochs = 300, verbose=0, class_weight = {0:7, 1:1})
    
# evaluate the model                    
y_pred =  model.predict(X_test_rfe)
y_pred = y_pred > 0.5

s = precision_score(y_test_2, y_pred)
print("\n\nprecision\n", s)
print("recall\n", recall_score(y_test_2, y_pred))




precision
 0.0
recall
 0.0


In [ ]:
y_pred

In [ ]:

rfe =  RFE(RandomForestClassifier(n_estimators=100), 5).fit(X_train, y_train)

X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)
    
sm = SMOTE(random_state=27)
X_train_res, y_train_res = sm.fit_sample(X_train_rfe, y_train.ravel())

# create model
n_cols = X_train_res.shape[1]
model = Sequential()
    
model.add(Dense(7, activation='elu', input_dim = n_cols))
model.add(Dropout(0.5))

model.add(Dense(5, activation='elu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='elu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')


# Fit the model
model.fit(X_train_res, y_train_res, epochs = 250, verbose=0, class_weight = {0:1, 1:1.7})
    
# evaluate the model                    
y_pred =  model.predict(X_test_rfe)
y_pred = y_pred > 0.5

s = precision_score(y_pred,y_test)
print("\n\nprecision\n", s)
print("recall\n", recall_score(y_pred,y_test))
